### Modular python script of SR-GNN

In [1]:
%conda info


     active environment : fgenv
    active env location : C:\Users\Sikdara\.conda\envs\fgenv
            shell level : 2
       user config file : C:\Users\Sikdara\.condarc
 populated config files : C:\Users\Sikdara\.condarc
          conda version : 22.9.0
    conda-build version : 3.22.0
         python version : 3.9.13.final.0
       virtual packages : __cuda=11.7=0
                          __win=0=0
                          __archspec=1=x86_64
       base environment : C:\ProgramData\Anaconda3  (read only)
      conda av data dir : C:\ProgramData\Anaconda3\etc\conda
  conda av metadata url : None
           channel URLs : https://repo.anaconda.com/pkgs/main/win-64
                          https://repo.anaconda.com/pkgs/main/noarch
                          https://repo.anaconda.com/pkgs/r/win-64
                          https://repo.anaconda.com/pkgs/r/noarch
                          https://repo.anaconda.com/pkgs/msys2/win-64
                          https://repo.anaconda.c

In [3]:
import tensorflow as tf
from platform import python_version

print("Num GPUs Available --> ", len(tf.config.list_physical_devices('GPU')))
print(" ")
print('tensorflow version -->',tf.__version__)
print(" ")
print('python version -->', python_version())

Num GPUs Available -->  1
 
tensorflow version --> 2.10.0
 
python version --> 3.9.16


In [1]:
from utils import crop

In [4]:
import os

os.path.realpath(__file__)

NameError: name '__file__' is not defined

### Importing data parameter

In [17]:
import yaml, json

with open('config.yaml', 'r') as file:
    param = yaml.load(file, Loader=yaml.FullLoader)
param

{'DATA': {'nb_classes': 100,
  'image_size': [224, 224],
  'dataset_dir': '/mnt/DataStore/projects_share/datasets/Aircraft'},
 'HARDWARE': {'multi_gpu': False, 'gpu_id': -1, 'gpu_utilisation': 1},
 'AUGMENTATION': {'aug_zoom': 0.15,
  'aug_tx': 0,
  'aug_ty': 0,
  'aug_rotation': 15},
 'MODEL': {'batch_size': 16, 'learning_rate': 0.01, 'model_name': 'model'},
 'TRAIN': {'validation_freq': 5,
  'checkpoint_freq': 50,
  'epochs': 200,
  'alpha_teleport': 0.4,
  'channels': '512_test',
  'learning_rate': 0.01}}

In [31]:
 print('parameter configuration: \n', json.dumps(param, sort_keys = True, indent = 3))

parameter configuration: 
 {
   "AUGMENTATION": {
      "aug_rotation": 15,
      "aug_tx": 0,
      "aug_ty": 0,
      "aug_zoom": 0.15
   },
   "DATA": {
      "dataset_dir": "/mnt/DataStore/projects_share/datasets/Aircraft",
      "image_size": [
         224,
         224
      ],
      "nb_classes": 100
   },
   "HARDWARE": {
      "gpu_id": -1,
      "gpu_utilisation": 1,
      "multi_gpu": false
   },
   "MODEL": {
      "batch_size": 16,
      "learning_rate": 0.01,
      "model_name": "model"
   },
   "TRAIN": {
      "alpha_teleport": 0.4,
      "channels": "512_test",
      "checkpoint_freq": 50,
      "epochs": 200,
      "learning_rate": 0.01,
      "validation_freq": 5
   }
}


In [11]:
# << data parameters >>
nb_classes = param['DATA']['nb_classes']
image_size = tuple( param['DATA']['image_size'] )
dataset_dir = param['DATA']['dataset_dir']

# << hardware parameters >>   ~~~~~~~~~~~~~~~~~~~
multi_gpu = param['HARDWARE']['multi_gpu']
gpu_id = param['HARDWARE']['gpu_id']
gpu_utilisation = param['HARDWARE']['gpu_utilisation']

# << augmentation parameters >>
aug_zoom = param['AUGMENTATION']['aug_zoom']
aug_tx = param['AUGMENTATION']['aug_tx']
aug_ty = param['AUGMENTATION']['aug_ty']
aug_rotation= param['AUGMENTATION']['aug_rotation']

# << model parameters >>    ~~~~~~~~~~~~~~~~~~~
batch_size =  param['MODEL']['batch_size']
lr = param['MODEL']['learning_rate']
model_name = param['MODEL']['model_name']

# << training parameters >>
validation_freq = param['TRAIN']['validation_freq']
checkpoint_freq = param['TRAIN']['checkpoint_freq']
epochs = param['TRAIN']['epochs']

In [12]:
print('nb_classes --> ', nb_classes, type(nb_classes))
print('image_size --> ', image_size, type(image_size))
print('dataset_dir --> ', dataset_dir, type(dataset_dir))
print(' ')
print('multi_gpu --> ', multi_gpu, type(multi_gpu))
print('gpu_id --> ', gpu_id, type(gpu_id))
print('gpu_utilisation --> ', gpu_utilisation, type(gpu_utilisation))
print(' ')
print('aug_zoom --> ', aug_zoom, type(aug_zoom))
print('aug_ty --> ', aug_ty, type(aug_ty))
print(' ')
print('batch_size --> ', batch_size, type(batch_size))
print('learning_rate --> ', lr, type(lr))
# print('optimizer --> ', optimizer, type(optimizer))
# print('model --> ', model, type(model))
print(' ')
print('validation_freq --> ', validation_freq, type(validation_freq))
print('checkpoint_freq --> ', checkpoint_freq, type(checkpoint_freq))
print('epochs --> ', epochs, type(epochs))
# print('alpha_teleport --> ', alpha_teleport, type(alpha_teleport))
# print('channels --> ', channels, type(channels))

nb_classes -->  100 <class 'int'>
image_size -->  (224, 224) <class 'tuple'>
dataset_dir -->  /mnt/DataStore/projects_share/datasets/Aircraft <class 'str'>
 
multi_gpu -->  False <class 'bool'>
gpu_id -->  -1 <class 'int'>
gpu_utilisation -->  1 <class 'int'>
 
aug_zoom -->  0.15 <class 'float'>
aug_ty -->  0 <class 'int'>
 
batch_size -->  16 <class 'int'>
learning_rate -->  0.01 <class 'float'>
 
validation_freq -->  5 <class 'int'>
checkpoint_freq -->  50 <class 'int'>
epochs -->  200 <class 'int'>


In [2]:
!nvidia-smi

/bin/bash: /home/sikdara/anaconda3/envs/fgenv/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Fri Jun 16 09:58:52 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Quadro RTX 8000     Off  | 00000000:3B:00.0 Off |                    0 |
| N/A   56C    P0    68W / 250W |  14962MiB / 46080MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+-------------

In [1]:
import ast

# =============== load and compile ~DEFAULT CONFIGURATION~ parameters
param_dir = "parameters.txt"
with  open(param_dir, "r") as paramf:
    params = paramf.readlines()

# << data parameters >>
nb_classes = int(params[1].split(":")[-1].rstrip())
image_size = ast.literal_eval(params[2].split(":")[-1].rstrip())
dataset_dir = params[3].split(":")[-1].rstrip()

# << hardware parameters >>   ~~~~~~~~~~~~~~~~~~~
multi_gpu = ast.literal_eval(params[6].split(":")[-1].rstrip())
gpu_id = params[7].split(":")[-1].rstrip()
gpu_utilisation = float(params[8].split(":")[-1].rstrip())

# << augmentation parameters >>
aug_zoom = float(params[11].split(":")[-1].rstrip())
aug_tx = int(params[12].split(":")[-1].rstrip())
aug_ty = int(params[13].split(":")[-1].rstrip())
aug_rotation= int(params[14].split(":")[-1].rstrip())

# << model parameters >>    ~~~~~~~~~~~~~~~~~~~
batch_size = int(params[17].split(":")[-1].rstrip())
lr = float(params[18].split(":")[-1].rstrip())
#optimizer = eval(params[19].split(":")[-1].rstrip())
model_name = params[20].split(":")[-1].rstrip()

# << training parameters >>
validation_freq = int(params[23].split(":")[-1].rstrip())
checkpoint_freq = int(params[24].split(":")[-1].rstrip())
epochs = int(params[25].split(":")[-1].rstrip())

In [2]:
print('nb_classes --> ', nb_classes, type(nb_classes))
print('image_size --> ', image_size, type(image_size))
print('dataset_dir --> ', dataset_dir, type(dataset_dir))
print(' ')
print('multi_gpu --> ', multi_gpu, type(multi_gpu))
print('gpu_id --> ', gpu_id, type(gpu_id))
print('gpu_utilisation --> ', gpu_utilisation, type(gpu_utilisation))
print(' ')
print('aug_zoom --> ', aug_zoom, type(aug_zoom))
print('aug_ty --> ', aug_ty, type(aug_ty))
print(' ')
print('batch_size --> ', batch_size, type(batch_size))
print('learning_rate --> ', lr, type(lr))
# print('optimizer --> ', optimizer, type(optimizer))
# print('model --> ', model, type(model))
print(' ')
print('validation_freq --> ', validation_freq, type(validation_freq))
print('checkpoint_freq --> ', checkpoint_freq, type(checkpoint_freq))
print('epochs --> ', epochs, type(epochs))
# print('alpha_teleport --> ', alpha_teleport, type(alpha_teleport))
# print('channels --> ', channels, type(channels))

nb_classes -->  100 <class 'int'>
image_size -->  (224, 224) <class 'tuple'>
dataset_dir -->  /mnt/DataStore/projects_share/datasets/Aircraft <class 'str'>
 
multi_gpu -->  False <class 'bool'>
gpu_id -->   -1 <class 'str'>
gpu_utilisation -->  1.0 <class 'float'>
 
aug_zoom -->  0.15 <class 'float'>
aug_ty -->  0 <class 'int'>
 
batch_size -->  16 <class 'int'>
learning_rate -->  0.01 <class 'float'>
 
validation_freq -->  5 <class 'int'>
checkpoint_freq -->  50 <class 'int'>
epochs -->  200 <class 'int'>


In [4]:
params

['#~~~~~~~~~~~~~~~~~~~     data parameters     ~~~~~~~~~~~~~~~~~~~\n',
 'nb_classes:100\n',
 'image_size:(224,224)\n',
 'dataset_dir:/mnt/DataStore/projects_share/datasets/Aircraft\n',
 '\n',
 '#~~~~~~~~~~~~~~~~~~~   hardware parameters   ~~~~~~~~~~~~~~~~~~~\n',
 'multi_gpu:False\n',
 'gpu_id: -1\n',
 'gpu_utilisation:1\n',
 '\n',
 '#~~~~~~~~~~~~~~~~~~~ augmentation parameters ~~~~~~~~~~~~~~~~~~~\n',
 'aug_zoom:0.15\n',
 'aug_tx:0\n',
 'aug_ty:0\n',
 'aug_rotation:15\n',
 '\n',
 '#~~~~~~~~~~~~~~~~~~~    model parameters     ~~~~~~~~~~~~~~~~~~~\n',
 'batch_size:16\n',
 'learning_rate:0.01\n',
 'optimizer:SGD(lr=0.01)\n',
 'model_name:model\n',
 '\n',
 '#~~~~~~~~~~~~~~~~~~~   training parameters   ~~~~~~~~~~~~~~~~~~~\n',
 'validation_freq:5\n',
 'checkpoint_freq:50\n',
 'epochs:200\n',
 'alpha_teleport:0.40 \n',
 'channels:512_test\n',
 'learning_rate:0.01']

In [6]:
# import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

# Create a tensor
tensor = tf.placeholder(tf.float32, shape=(None, 42, 42, 2048))

# Convert to (None, 2048, 1764)
transposed_tensor = tf.transpose(tensor, perm=[0, 3, 1, 2])
reshaped_tensor = tf.reshape(transposed_tensor, shape=(-1, 2048, 1764))

# Convert back to (None, 42, 42, 2048)
restored_tensor = tf.transpose(reshaped_tensor, perm=[0, 2, 1])
reshaped_back_tensor = tf.reshape(restored_tensor, shape=(-1, 42, 42, 2048))

# Print the shapes of original and restored tensors
print("Original tensor shape:", tensor.shape)
print("Original tensor shape:", reshaped_tensor.shape)
print("Restored tensor shape:", reshaped_back_tensor.shape)


Original tensor shape: (?, 42, 42, 2048)
Original tensor shape: (?, 2048, 1764)
Restored tensor shape: (?, 42, 42, 2048)


In [7]:
42 * 42

1764

In [96]:
import tensorflow as tf
import numpy as np

# Assume you have a tensor with shape [batch_size, height, width, channels]
tensor = tf.random.normal([4, 2048, 42, 42])

expanded_tensor = tf.expand_dims(tensor, axis = 1)
# print(expanded_tensor.shape)

# Split the tensor along the channel dimension (axis=3)
splits = tf.split(tensor, num_or_size_splits=4, axis=1)
print(np.shape(splits))

# The result is a list of tensors
# Each tensor has shape [batch_size, height, width, split_channels]
sp_all = []
for split in splits:
    print('-----------')
    print(split.shape)
    a = tf.reshape(split, shape=(-1, 4, (2048//4) * 42 * 42  ))
    sp_all.append(a)
    print(a.shape)


NotImplementedError: Cannot convert a symbolic tf.Tensor (split_9:0) to a numpy array. This error may indicate that you're trying to pass a Tensor to a NumPy call, which is not supported.

In [58]:
np.shape(sp_all)

(4, 1, 4, 903168)

In [65]:
b = tf.reshape(expanded_tensor, shape = (-1, 4, (2048//4) * 42 * 42))
b.shape

TensorShape([4, 4, 903168])

In [75]:
b[0][-1]

<tf.Tensor: shape=(903168,), dtype=float32, numpy=
array([-2.25943   ,  1.6523793 , -0.99794465, ...,  1.4130237 ,
       -1.2022934 ,  1.014423  ], dtype=float32)>

In [76]:
sp_all[0][0][-1]

<tf.Tensor: shape=(903168,), dtype=float32, numpy=
array([-1.9232527 , -0.47618198, -1.4552858 , ...,  1.3851614 ,
       -0.26174375,  0.7149143 ], dtype=float32)>

In [46]:
import tensorflow as tf
import numpy as np

# Create a tensor with shape [batch_size, channels, height, width]
tensor = tf.random.normal([4, 16, 32, 32])

# Split the tensor into four parts along the channel dimension (axis=1)
splits = tf.split(tensor, num_or_size_splits=4, axis=1)

# Flatten each split
flattened_splits = [tf.reshape(split, [split.shape[0], -1]) for split in splits]

for i, split in enumerate(flattened_splits):
    print("Flattened split", i+1, "shape:", split.shape)


Flattened split 1 shape: (4, 4096)
Flattened split 2 shape: (4, 4096)
Flattened split 3 shape: (4, 4096)
Flattened split 4 shape: (4, 4096)


In [50]:
np.shape(splits)

(4, 4, 4, 32, 32)

In [144]:
import tensorflow as tf
import numpy as np


cnodes = 8
base_dims = 2048


# Assume you have a tensor with shape [batch_size, channels, height, width]
tensor = tf.random.normal([4, 42, 42, 2048])
transposed_tensor = tf.transpose(tensor, perm=[0, 3, 1, 2])
print(transposed_tensor)
print(' ')


# Split into cnodes components
splits = tf.split(transposed_tensor, num_or_size_splits = cnodes, axis=1)
print(splits, len(splits))
print(' ')

# Flatten each split
flattened_splits = [tf.expand_dims( tf.reshape(split, [split.shape[0], -1]), 1 ) for split in splits]
print(flattened_splits, len(flattened_splits))
print(' ')

# Concatenate into single tensor
joined = tf.concat(flattened_splits, 1)
print(joined)
print(' ')

# Apply channel wise gcn


# Restoring back to original tensor
restore_tensor = [tf.reshape(x, shape=(-1, base_dims//cnodes, 42, 42))   for x in tf.split(joined, num_or_size_splits = cnodes, axis=1)]
print(restore_tensor, len(restore_tensor))
print(' ')

restore_tensor = tf.concat(restore_tensor, 1)
print(restore_tensor.shape)

# Permute back to Channel last
restore_tensor = tf.transpose(restore_tensor, perm = [0, 2, 3, 1])
print(restore_tensor)


Tensor("transpose_7:0", shape=(4, 2048, 42, 42), dtype=float32)
 
[<tf.Tensor 'split_57:0' shape=(4, 256, 42, 42) dtype=float32>, <tf.Tensor 'split_57:1' shape=(4, 256, 42, 42) dtype=float32>, <tf.Tensor 'split_57:2' shape=(4, 256, 42, 42) dtype=float32>, <tf.Tensor 'split_57:3' shape=(4, 256, 42, 42) dtype=float32>, <tf.Tensor 'split_57:4' shape=(4, 256, 42, 42) dtype=float32>, <tf.Tensor 'split_57:5' shape=(4, 256, 42, 42) dtype=float32>, <tf.Tensor 'split_57:6' shape=(4, 256, 42, 42) dtype=float32>, <tf.Tensor 'split_57:7' shape=(4, 256, 42, 42) dtype=float32>] 8
 
[<tf.Tensor 'ExpandDims_189:0' shape=(4, 1, 451584) dtype=float32>, <tf.Tensor 'ExpandDims_190:0' shape=(4, 1, 451584) dtype=float32>, <tf.Tensor 'ExpandDims_191:0' shape=(4, 1, 451584) dtype=float32>, <tf.Tensor 'ExpandDims_192:0' shape=(4, 1, 451584) dtype=float32>, <tf.Tensor 'ExpandDims_193:0' shape=(4, 1, 451584) dtype=float32>, <tf.Tensor 'ExpandDims_194:0' shape=(4, 1, 451584) dtype=float32>, <tf.Tensor 'ExpandDims

In [122]:
splits

[<tf.Tensor 'split_20:0' shape=(4, 64, 42, 42) dtype=float32>,
 <tf.Tensor 'split_20:1' shape=(4, 64, 42, 42) dtype=float32>,
 <tf.Tensor 'split_20:2' shape=(4, 64, 42, 42) dtype=float32>,
 <tf.Tensor 'split_20:3' shape=(4, 64, 42, 42) dtype=float32>,
 <tf.Tensor 'split_20:4' shape=(4, 64, 42, 42) dtype=float32>,
 <tf.Tensor 'split_20:5' shape=(4, 64, 42, 42) dtype=float32>,
 <tf.Tensor 'split_20:6' shape=(4, 64, 42, 42) dtype=float32>,
 <tf.Tensor 'split_20:7' shape=(4, 64, 42, 42) dtype=float32>,
 <tf.Tensor 'split_20:8' shape=(4, 64, 42, 42) dtype=float32>,
 <tf.Tensor 'split_20:9' shape=(4, 64, 42, 42) dtype=float32>,
 <tf.Tensor 'split_20:10' shape=(4, 64, 42, 42) dtype=float32>,
 <tf.Tensor 'split_20:11' shape=(4, 64, 42, 42) dtype=float32>,
 <tf.Tensor 'split_20:12' shape=(4, 64, 42, 42) dtype=float32>,
 <tf.Tensor 'split_20:13' shape=(4, 64, 42, 42) dtype=float32>,
 <tf.Tensor 'split_20:14' shape=(4, 64, 42, 42) dtype=float32>,
 <tf.Tensor 'split_20:15' shape=(4, 64, 42, 42) dt

In [95]:
flattened_splits

[<tf.Tensor 'ExpandDims:0' shape=(4, 1, 903168) dtype=float32>,
 <tf.Tensor 'ExpandDims_1:0' shape=(4, 1, 903168) dtype=float32>,
 <tf.Tensor 'ExpandDims_2:0' shape=(4, 1, 903168) dtype=float32>,
 <tf.Tensor 'ExpandDims_3:0' shape=(4, 1, 903168) dtype=float32>]

In [102]:
tf.reshape(joined[1], shape=(-1, 512, 42, 42)) 

<tf.Tensor 'Reshape_15:0' shape=(4, 512, 42, 42) dtype=float32>

In [107]:
splits2 = tf.split(joined, num_or_size_splits = cnodes, axis=1)

In [110]:
tf.reshape(splits2[1], shape=(-1, 512, 42, 42)) 

<tf.Tensor 'Reshape_24:0' shape=(4, 512, 42, 42) dtype=float32>

In [123]:
restore_tensor

<tf.Tensor 'concat_9:0' shape=(4, 2048, 42, 42) dtype=float32>

In [117]:
tensor

<tf.Tensor 'random_normal_15:0' shape=(4, 2048, 42, 42) dtype=float32>

In [119]:
2048/32

64.0